# Evaluation

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import regex as re

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pwd

/content


In [4]:
# Nhập dữ liệu
with open('/content/drive/MyDrive/ViNLU/dev-results/seq.out', 'r', encoding= 'utf-8') as file:
    out_lines = file.readlines()
    file.close()

with open('/content/drive/MyDrive/ViNLU/dev-results/seq.in', 'r', encoding= 'utf-8') as file:
    in_lines = file.readlines()
    file.close()

with open('/content/drive/MyDrive/ViNLU/dev-results/label', 'r', encoding= 'utf-8') as file:
    out_labels = file.readlines()
    file.close()

with open('/content/drive/MyDrive/ViNLU/dev-results/predict.out', 'r', encoding= 'utf-8') as file:
    test_lines = file.readlines()
    file.close()

In [5]:
# Số samples trong bộ test
len(out_lines), len(test_lines)

(500, 500)

In [6]:
# Chuẩn hóa
out_lines = [x.strip() for x in out_lines]
in_lines = [x.strip() for x in in_lines]
out_labels = [x.strip() for x in out_labels]
test_lines = [x.strip() for x in test_lines]

In [7]:
out_lines[0]

'O O O O B-fromloc.city_name O B-depart_time.time I-depart_time.time I-depart_time.time I-depart_time.time I-depart_time.time O O B-toloc.city_name O B-arrive_time.period_of_day I-arrive_time.period_of_day O B-arrive_time.time I-arrive_time.time I-arrive_time.time I-arrive_time.time'

In [8]:
test_lines[0]

'<flight> -> tôi muốn bay từ [hải_phòng:B-fromloc.city_name] lúc [8:B-depart_time.time] [giờ:I-depart_time.time] [38:I-depart_time.time] [phút:I-depart_time.time] [sáng:I-depart_time.time] và đến [huế:B-toloc.city_name] vào [buổi:B-arrive_time.period_of_day] [sáng:I-arrive_time.period_of_day] lúc [11:B-arrive_time.time] [giờ:I-arrive_time.time] [10:I-arrive_time.time] [phút:I-arrive_time.time]'

In [9]:
# Chuyển định dạng
def test2out(test_line):
    words = test_line.split()
    intent = words[0]
    words = words[2:]
    str = []
    for word in words:
        if word[0] != '[':
            str.append('O')
        else:
            info = word[1:-1]
            str.append(info.split(':')[-1])
    slots = ' '.join(str)
    return intent[1:-1], slots

## Khảo sát intent

In [10]:
# Kiểm tra
intent, slots = test2out(test_lines[0])
intent, out_labels[0]

('flight', 'flight')

In [11]:
diff_ids = []
for i in range(len(test_lines)):
  label, gt_label = test2out(test_lines[i])[0], out_labels[i]
  if label != gt_label:
    diff_ids.append(i)

print(len(diff_ids))

11


In [12]:
# Các lỗi gặp phải
diffs = []
for i in diff_ids:
  label, gt_label = test2out(test_lines[i])[0], out_labels[i]
  diffs.append(f'{label}||{gt_label}')

diffs = {i:diffs.count(i) for i in set(diffs)}
diffs = dict(sorted(diffs.items(), key=lambda item: -item[1]))
diffs # test||out

{'flight||airfare#flight': 2,
 'airfare||airfare#flight_time': 2,
 'airfare||abbreviation': 1,
 'flight||airfare': 1,
 'aircraft||aircraft#flight': 1,
 'airfare#flight||flight': 1,
 'flight||airport': 1,
 'airfare||flight': 1,
 'flight||flight_time': 1}

In [13]:
# Các câu chứa lỗi intent
[in_lines[i] for i in diff_ids]

['trên các chuyến bay của bamboo airways bay từ hải_phòng đến tây an trước 10 giờ sáng loại máy_bay nào được sử_dụng',
 'cho tôi danh_sách các chuyến bay vào ngày 27 tháng 12 từ đài_bắc đến singapore và giá vé tương_ứng',
 'cho tôi biết các chuyến bay với giá vé qx giữa cà_mau và tokyo của vietnam airlines',
 'hiển_thị cho tôi giá và thời_gian cho chuyến bay hạng nhất từ hà_nội đến đà_lạt vào mùa hè tới',
 'các chuyến bay từ hải_phòng đến moscow dừng ở đâu',
 'mã giá vé nào áp_dụng cho các chuyến bay từ huế đến phú_quốc',
 'liệt_kê các hạng ghế khác nhau mà một hãng hàng_không cung_cấp',
 'khi nào máy_bay rời hải_phòng',
 'cho tôi xem tất_cả các chuyến bay qua đêm từ washington dc đến madrid và liệt_kê giá vé của chúng',
 'cho tôi biết chi_phí và thời_gian của các chuyến bay từ phú_quốc đến cam_ranh',
 'tôi muốn bay từ hải_phòng đến cà_mau tôi muốn có giá vé rẻ nhất']

### Khảo sát intent logits

In [14]:
# Load logits
intent_logits = np.loadtxt('/content/drive/MyDrive/ViNLU/dev-results/intent_logits.txt')
intent_logits.shape

(500, 25)

In [16]:
# Confidence
confidences = np.max(intent_logits, axis= 1)
np.array(sorted(confidences)[:45]) # Đặt threshold = 0.98

array([0.77948558, 0.79899025, 0.82206768, 0.92434824, 0.96028531,
       0.9927246 , 0.99530113, 0.99773008, 0.99818939, 0.99831772,
       0.99892324, 0.99944764, 0.99946445, 0.99958175, 0.99966669,
       0.99969244, 0.99973732, 0.99975425, 0.9998253 , 0.99982733,
       0.99983215, 0.99983668, 0.99985468, 0.99988198, 0.99989069,
       0.99989355, 0.99989867, 0.99990165, 0.99990427, 0.99990749,
       0.99991262, 0.99991405, 0.99992013, 0.99992192, 0.99992418,
       0.99993014, 0.99993038, 0.99994481, 0.99995017, 0.99995089,
       0.99995232, 0.99995565, 0.99995577, 0.9999572 , 0.99995732])

In [17]:
hard_ids = []
for i in range(intent_logits.shape[0]):
  if i in diff_ids:
    continue
  if confidences[i] < 0.98:
    hard_ids.append(i)

hard_ids = sorted(hard_ids, key= lambda x:confidences[x])
for id in hard_ids:
  print(id, confidences[id], out_labels[id])

336 0.7794855833053589 ground_fare
454 0.9243482351303101 airfare#flight


In [18]:
# Các câu model dự đoán ĐÚNG nhưng intent confidence THẤP. Các câu được sắp xếp theo thứ tự confidences tăng dần
[in_lines[i] for i in hard_ids]

['và chi_phí để đi từ sân_bay bournemouth đến trung_tâm thành_phố',
 'liệt_kê giúp tôi các chuyến bay và giá vé tương_ứng của tất_cả các hãng hàng_không từ portsmouth đến hà_nội']

## Khảo sát slot filling

In [19]:
# Kiểm tra
intent, slots = test2out(test_lines[0])
slots

'O O O O B-fromloc.city_name O B-depart_time.time I-depart_time.time I-depart_time.time I-depart_time.time I-depart_time.time O O B-toloc.city_name O B-arrive_time.period_of_day I-arrive_time.period_of_day O B-arrive_time.time I-arrive_time.time I-arrive_time.time I-arrive_time.time'

In [20]:
out_lines[0]

'O O O O B-fromloc.city_name O B-depart_time.time I-depart_time.time I-depart_time.time I-depart_time.time I-depart_time.time O O B-toloc.city_name O B-arrive_time.period_of_day I-arrive_time.period_of_day O B-arrive_time.time I-arrive_time.time I-arrive_time.time I-arrive_time.time'

In [21]:
# Ghi file
outpath = 'slot_diff.txt'
lines = ['id||seq_test||seq_out\n']
raw_lines = ['id||output||ground_truth\n']

for i in range(len(out_lines)):
    intent, slots = test2out(test_lines[i])
    if slots != out_lines[i]:
        lines.append('||'.join([str(i), slots, out_lines[i]]) + '\n')
        raw_lines.append('||'.join([str(i), slots, out_lines[i]]))

with open(outpath, 'w', encoding= 'utf-8') as file:
    file.writelines(lines)
    file.close()

raw_lines

['id||output||ground_truth\n',
 '10||O O O O B-fromloc.city_name I-fromloc.city_name O B-toloc.city_name||O O O O B-fromloc.city_name I-fromloc.city_name O B-toloc.state_name',
 '14||O O O O B-fromloc.city_name O B-toloc.city_name I-toloc.city_name I-toloc.city_name O B-depart_time.time_relative B-depart_time.time I-depart_time.time I-depart_time.time O B-airline_name I-airline_name||O O O O B-fromloc.city_name O B-toloc.airport_name I-toloc.airport_name I-toloc.airport_name O B-depart_time.time_relative B-depart_time.time I-depart_time.time I-depart_time.time O B-airline_name I-airline_name',
 '16||O O O O O O O O B-depart_time.period_of_day B-depart_time.period_mod O B-fromloc.city_name O B-toloc.city_name O B-depart_date.day_number I-depart_date.day_number B-depart_date.month_name I-depart_date.month_name O O O B-arrive_time.period_mod O B-return_time.period_of_day B-return_date.day_number I-return_date.day_number B-return_date.month_name I-airport_name||O O O O O O O O B-depart_tim

In [22]:
# Các lỗi gặp phải
diffs = []
slot_diff_ids = []
for t in range(len(lines)):
    words = [x.strip() for x in lines[t].split('||')]
    if words[0] != 'id':
      id = int(words[0])
      slot_diff_ids.append(id)

    test = words[1].split()
    out = words[2].split()

    for i in range(len(test)):
        if test[i] != out[i]:
            diffs.append(f'{test[i]}||{out[i]}')

diffs = {i:diffs.count(i) for i in set(diffs)}
diffs = dict(sorted(diffs.items(), key=lambda item: -item[1]))
diffs # test||out

{'I-airport_name||I-fromloc.airport_name': 4,
 'B-depart_time.period_of_day||B-arrive_time.period_of_day': 4,
 'B-arrive_date.day_name||B-depart_date.day_name': 3,
 'B-flight_mod||O': 3,
 'B-toloc.city_name||B-toloc.state_name': 3,
 'I-flight_mod||O': 2,
 'I-arrive_date.day_name||I-depart_date.day_name': 2,
 'B-airport_name||B-fromloc.airport_name': 2,
 'I-airline_name||B-airline_name': 2,
 'I-toloc.city_name||I-toloc.airport_name': 2,
 'B-airline_name||O': 2,
 'B-arrive_time.period_mod||O': 2,
 'B-toloc.city_name||B-city_name': 2,
 'O||B-flight_mod': 2,
 'I-depart_time.period_of_day||I-arrive_time.period_of_day': 2,
 'O||B-flight_time': 2,
 'B-toloc.city_name||B-fromloc.city_name': 2,
 'I-transport_type||B-transport_type': 2,
 'I-fromloc.city_name||I-fromloc.airport_name': 2,
 'I-airline_name||O': 1,
 'I-connect||O': 1,
 'B-arrive_date.day_number||B-depart_date.day_number': 1,
 'B-toloc.city_name||B-toloc.airport_name': 1,
 'B-toloc.city_name||B-toloc.airport_code': 1,
 'B-return_date

In [23]:
# Các câu bị sai slot
[in_lines[i] for i in slot_diff_ids]

['các chuyến bay từ thành_phố hồ_chí_minh đến florida',
 'các chuyến bay từ hà_nội đến phố_đông thượng hải khởi_hành sau 7 rưỡi sáng của vietnam airlines',
 'tôi muốn đặt trước một chuyến bay vào sáng sớm từ tuy_hoà đến điện_biên_phủ vào ngày 22 tháng 2 và trở về muộn vào tối ngày 22 tháng 2',
 'ok tôi sẽ thử lần cuối , cho tôi biết loại máy_bay mà vietnam airlines sử_dụng cho chuyến bay từ huế đến phú_quốc vào buổi sáng trước 10 giờ',
 'tôi muốn có một chuyến bay giá rẻ từ norwich đến hà_nội vào ngày mùng 3 tháng 1',
 'cho tôi biết thời_gian khởi_hành của một chuyến bay từ cà_mau đến hải_phòng vào buổi chiều',
 'cho tôi xem các chuyến bay từ côn_đảo đến quảng_ninh',
 'tìm cho tôi chuyến bay khởi_hành sớm nhất tại hải_phòng và trở về muộn nhất ở quảng châu để tôi có_thể có_mặt ở quảng châu lâu nhất và trở về hải_phòng vào cùng ngày',
 'mất bao_lâu để đi từ sân_bay quy nhơn đến trung_tâm thành_phố quy nhơn',
 'cho tôi xem lịch_trình phương_tiện giao_thông đường_bộ ở cam_ranh vào buổi sá